<a href="https://colab.research.google.com/github/bala-ceg/backtest-strategies/blob/main/weeklyexpiry_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install & Import Necessary Libs

In [1]:
!pip install nsepython

In [2]:
from nsepython import *

## NIFTY OI sample snippet

In [3]:
symbol = "NIFTY"
start_date = "07-06-2024"
end_date ="12-06-2024"
instrumentType = "options"
expiry_date ="13-Jun-2024"
put_df = derivative_history(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice="",optionType="PE")
call_df = derivative_history(symbol,start_date,end_date,instrumentType,expiry_date,strikePrice="",optionType="CE")
if not put_df.empty:
  put_df['FH_OPEN_INT'] = put_df['FH_OPEN_INT'].astype(int)
  put_df['FH_TOT_TRADED_QTY'] = put_df['FH_TOT_TRADED_QTY'].astype(int)
  put_df = put_df.sort_values(by=['FH_OPEN_INT'], ascending=False)
  underlying_value = put_df['FH_UNDERLYING_VALUE'].iloc[0]

if not call_df.empty:
  call_df['FH_OPEN_INT'] = call_df['FH_OPEN_INT'].astype(int)
  call_df['FH_TOT_TRADED_QTY'] = call_df['FH_TOT_TRADED_QTY'].astype(int)
  call_df = call_df.sort_values(by=['FH_OPEN_INT'], ascending=False)
  underlying_value = call_df['FH_UNDERLYING_VALUE'].iloc[0]

In [4]:
put_total_open_int = put_df['FH_OPEN_INT'].sum()
put_total_volume = put_df['FH_TOT_TRADED_QTY'].sum()

put_df['FH_OPEN_INT_PERCENTAGE'] = (put_df['FH_OPEN_INT'] / put_total_open_int) * 100

put_df[['FH_STRIKE_PRICE', 'FH_OPEN_INT','FH_OPEN_INT_PERCENTAGE']].head(5)

,FH_STRIKE_PRICE,FH_OPEN_INT,FH_OPEN_INT_PERCENTAGE
12,23000.00,8672575,8.213866
16,22800.00,6317300,5.983166
8,23200.00,6306300,5.972747
22,22500.00,5639725,5.341429
32,22000.00,5141825,4.869864


In [5]:
put_df['FH_STRIKE_PRICE'] = put_df['FH_STRIKE_PRICE'].astype(float)
put_df['ABS_DIFF'] = abs(put_df['FH_STRIKE_PRICE'] - float(underlying_value))

sorted_put_df = put_df.sort_values(by='ABS_DIFF')

closest_rows = sorted_put_df[['FH_STRIKE_PRICE', 'FH_OPEN_INT','ABS_DIFF','FH_OPEN_INT_PERCENTAGE','FH_TOT_TRADED_QTY']]

print(closest_rows.head(3))
print(underlying_value)
put_df['FH_OPEN_INT'].idxmax()
row_with_max_open_int = put_df.loc[put_df['FH_OPEN_INT'].idxmax()]
row_with_max_open_int['FH_OPEN_INT']
near_strike_put_open_int = closest_rows['FH_OPEN_INT'].head(3).sum()
print(near_strike_put_open_int)
near_strike_put_vol = closest_rows['FH_TOT_TRADED_QTY'].head(3).sum()
print(near_strike_put_vol)

   FH_STRIKE_PRICE  FH_OPEN_INT  ABS_DIFF  FH_OPEN_INT_PERCENTAGE  \
6          23300.0      4479850     22.95                4.242902   
5          23350.0      2018600     27.05                1.911832   
7          23250.0      1960200     72.95                1.856521   

   FH_TOT_TRADED_QTY  
6          146726850  
5           97113150  
7           54860850  
23322.95
8458650
298700850


In [6]:
call_total_open_int = call_df['FH_OPEN_INT'].sum()
call_total_volume = call_df['FH_TOT_TRADED_QTY'].sum()

call_df['FH_OPEN_INT_PERCENTAGE'] = (call_df['FH_OPEN_INT'] / call_total_open_int) * 100

call_df[['FH_STRIKE_PRICE', 'FH_OPEN_INT','FH_OPEN_INT_PERCENTAGE']].head(5)


,FH_STRIKE_PRICE,FH_OPEN_INT,FH_OPEN_INT_PERCENTAGE
2,23500.00,7675275,18.055456
4,23400.00,6733200,15.839302
0,23600.00,5065475,11.916115
6,23300.00,3894175,9.160728
3,23450.00,3091550,7.272618


In [7]:
call_df['FH_STRIKE_PRICE'] = call_df['FH_STRIKE_PRICE'].astype(float)
call_df['ABS_DIFF'] = abs(call_df['FH_STRIKE_PRICE'] - float(underlying_value))

# Sort by the absolute difference
sorted_call_df = call_df.sort_values(by='ABS_DIFF')

# Select the rows that are closest to the underlying value
closest_rows = sorted_call_df[['FH_STRIKE_PRICE', 'FH_OPEN_INT','ABS_DIFF','FH_OPEN_INT_PERCENTAGE','FH_TOT_TRADED_QTY']]

print(closest_rows.head(3))
print(underlying_value)
call_df['FH_OPEN_INT'].idxmax()
row_with_max_open_int = call_df.loc[call_df['FH_OPEN_INT'].idxmax()]
row_with_max_open_int['FH_OPEN_INT']
near_strike_call_open_int = closest_rows['FH_OPEN_INT'].head(3).sum()
print(near_strike_call_open_int)
near_strike_call_vol = closest_rows['FH_TOT_TRADED_QTY'].head(3).sum()
print(near_strike_call_vol)

   FH_STRIKE_PRICE  FH_OPEN_INT  ABS_DIFF  FH_OPEN_INT_PERCENTAGE  \
6          23300.0      3894175     22.95                9.160728   
5          23350.0      2703625     27.05                6.360056   
7          23250.0       803825     72.95                1.890932   

   FH_TOT_TRADED_QTY  
6          104475950  
5          105632000  
7           21820550  
23322.95
7401625
231928500


In [8]:
pcr = float(put_total_open_int/call_total_open_int)
print("PCR OI:",pcr)
pcr_vol = float(put_total_volume/call_total_volume)
print("PCR volume:",pcr_vol)

PCR OI: 2.483790663017282
PCR volume: 1.6392114854111113


In [9]:
near_pcr = float(near_strike_put_open_int/near_strike_call_open_int)
print("Near PCR OI:",near_pcr)
near_pcr_vol = float(near_strike_put_vol/near_strike_call_vol)
print("Near PCR volume:",near_pcr_vol)

Near PCR OI: 1.142809855943796
Near PCR volume: 1.2879005814291904


In [10]:
call_df[['FH_STRIKE_PRICE', 'FH_OPEN_INT','ABS_DIFF','FH_OPEN_INT_PERCENTAGE']].head(15)

,FH_STRIKE_PRICE,FH_OPEN_INT,ABS_DIFF,FH_OPEN_INT_PERCENTAGE
2,23500.0,7675275,177.05,18.055456
4,23400.0,6733200,77.05,15.839302
0,23600.0,5065475,277.05,11.916115
6,23300.0,3894175,22.95,9.160728
3,23450.0,3091550,127.05,7.272618
5,23350.0,2703625,27.05,6.360056
1,23550.0,2661150,227.05,6.260137
12,23000.0,2198350,322.95,5.171438
8,23200.0,1852000,122.95,4.356678
16,22800.0,906675,522.95,2.132879


In [11]:
call_df[['FH_STRIKE_PRICE', 'FH_OPEN_INT']]

,FH_STRIKE_PRICE,FH_OPEN_INT
2,23500.0,7675275
4,23400.0,6733200
0,23600.0,5065475
6,23300.0,3894175
3,23450.0,3091550
...,...,...
59,20650.0,1400
53,20950.0,1200
47,21250.0,875
54,20900.0,775


In [12]:
import pandas as pd
historic_nifty_df = pd.read_csv('/content/Nifty50_5Ydata.csv')
historic_nifty_df['Date'] = pd.to_datetime(historic_nifty_df['Date'], format="%d-%m-%Y")
numeric_columns = ["Open", "High", "Low", "Price"]
for column in numeric_columns:
    historic_nifty_df[column] = historic_nifty_df[column].str.replace(",", "").astype(float)
historic_nifty_df['Change %'] = historic_nifty_df['Change %'].str.replace("%", "").astype(float)
historic_nifty_df

,Date,Price,Open,High,Low,Vol.,Change %
0,2024-06-20,23567.00,23586.15,23624.00,23442.60,280.34B,0.22
1,2024-06-19,23516.00,23629.85,23664.00,23412.90,328.81B,-0.18
2,2024-06-18,23557.90,23570.80,23579.05,23499.70,272.21B,0.39
3,2024-06-14,23465.60,23464.95,23490.40,23334.25,223.09B,0.29
4,2024-06-13,23398.90,23480.95,23481.05,23353.90,268.28B,0.33
...,...,...,...,...,...,...,...
1290,2019-04-04,11598.00,11660.20,11662.55,11559.20,349.03B,-0.39
1291,2019-04-03,11643.95,11735.30,11761.00,11629.15,365.76B,-0.59
1292,2019-04-02,11713.20,11711.55,11729.35,11655.85,386.13B,0.38
1293,2019-04-01,11669.15,11665.20,11738.10,11644.75,379.57B,0.39


In [13]:
data = historic_nifty_df.copy()
data['Day_of_Week'] = data['Date'].dt.dayofweek
data = data[['Date', 'Price', 'Day_of_Week','Change %']]
data['Week_Number'] = data['Date'].dt.isocalendar().week
data['Year'] = data['Date'].dt.isocalendar().year

day_map = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday'}
data['Day_of_Week'] = data['Day_of_Week'].map(day_map)

pivoted_data_1 = data.pivot_table(index=['Year', 'Week_Number'], columns='Day_of_Week', values='Price')
pivoted_data_2 = data.pivot_table(index=['Year', 'Week_Number'], columns='Day_of_Week', values='Change %')

In [14]:
import pandas as pd
from datetime import timedelta


start_date = pd.to_datetime("29-03-2019", format="%d-%m-%Y")
end_date = pd.to_datetime("20-06-2024", format="%d-%m-%Y")

start_list = []
end_list = []
target_list = []
years = []
week_numbers = []



current_date = start_date
while current_date <= end_date:
    if current_date.weekday() == 4:
        start_list.append(current_date.strftime("%d-%m-%Y"))
    if current_date.weekday() == 3:
        end_list.append(current_date.strftime("%d-%b-%Y"))
        years.append(current_date.isocalendar().year)
        week_numbers.append(current_date.isocalendar().week)
    if current_date.weekday() == 2:
        try :
          historic_nifty_df[historic_nifty_df['Date'] == current_date.strftime("%d-%m-%Y") ]['Price'].iloc[0]
          target_list.append(current_date.strftime("%d-%m-%Y"))
        except:
          previous_date = current_date - timedelta(days=1)
          target_list.append(previous_date.strftime("%d-%m-%Y"))

    current_date += timedelta(days=1)

print("Start List (Fridays):", start_list)
print("End List (Thursdays):", end_list)
print("Target List (Wednesdays):", target_list)
print(years)
print(week_numbers)

Start List (Fridays): ['29-03-2019', '05-04-2019', '12-04-2019', '19-04-2019', '26-04-2019', '03-05-2019', '10-05-2019', '17-05-2019', '24-05-2019', '31-05-2019', '07-06-2019', '14-06-2019', '21-06-2019', '28-06-2019', '05-07-2019', '12-07-2019', '19-07-2019', '26-07-2019', '02-08-2019', '09-08-2019', '16-08-2019', '23-08-2019', '30-08-2019', '06-09-2019', '13-09-2019', '20-09-2019', '27-09-2019', '04-10-2019', '11-10-2019', '18-10-2019', '25-10-2019', '01-11-2019', '08-11-2019', '15-11-2019', '22-11-2019', '29-11-2019', '06-12-2019', '13-12-2019', '20-12-2019', '27-12-2019', '03-01-2020', '10-01-2020', '17-01-2020', '24-01-2020', '31-01-2020', '07-02-2020', '14-02-2020', '21-02-2020', '28-02-2020', '06-03-2020', '13-03-2020', '20-03-2020', '27-03-2020', '03-04-2020', '10-04-2020', '17-04-2020', '24-04-2020', '01-05-2020', '08-05-2020', '15-05-2020', '22-05-2020', '29-05-2020', '05-06-2020', '12-06-2020', '19-06-2020', '26-06-2020', '03-07-2020', '10-07-2020', '17-07-2020', '24-07-2020

In [15]:
pcr_data_df = pd.DataFrame(columns=['Year', 'Week_Number', 'pcr', 'pcr_vol', 'near_pcr',  'near_pcr_vol'])
#pcr_data_df = pd.DataFrame()

for start_date, target_date, end_date,year,week_num in zip(start_list, target_list, end_list,years,week_numbers):
      symbol = "NIFTY"
      instrumentType = "options"
      print(start_date, target_date, end_date,year,week_num)
      put_df = derivative_history(symbol,start_date,target_date,instrumentType,end_date,strikePrice="",optionType="PE")
      call_df = derivative_history(symbol,start_date,target_date,instrumentType,end_date,strikePrice="",optionType="CE")
      try:
          underlying_value = put_df['FH_UNDERLYING_VALUE'].iloc[0]
          print(underlying_value)

          if not put_df.empty:
            put_df['FH_OPEN_INT'] = put_df['FH_OPEN_INT'].astype(int)
            put_df['FH_TOT_TRADED_QTY'] = put_df['FH_TOT_TRADED_QTY'].astype(int)
            put_df = put_df.sort_values(by=['FH_OPEN_INT'], ascending=False)
            put_total_open_int = put_df['FH_OPEN_INT'].sum()
            put_total_volume = put_df['FH_TOT_TRADED_QTY'].sum()
            put_df['FH_OPEN_INT_PERCENTAGE'] = (put_df['FH_OPEN_INT'] / put_total_open_int) * 100
            put_df['FH_STRIKE_PRICE'] = put_df['FH_STRIKE_PRICE'].astype(float)
            put_df['ABS_DIFF'] = abs(put_df['FH_STRIKE_PRICE'] - float(underlying_value))

            sorted_put_df = put_df.sort_values(by='ABS_DIFF')

            closest_rows = sorted_put_df[['FH_STRIKE_PRICE', 'FH_OPEN_INT','ABS_DIFF','FH_OPEN_INT_PERCENTAGE','FH_TOT_TRADED_QTY']]
            put_df['FH_OPEN_INT'].idxmax()
            row_with_max_open_int = put_df.loc[put_df['FH_OPEN_INT'].idxmax()]
            row_with_max_open_int['FH_OPEN_INT']
            near_strike_put_open_int = closest_rows['FH_OPEN_INT'].head(3).sum()
            near_strike_put_vol = closest_rows['FH_TOT_TRADED_QTY'].head(3).sum()

          if not call_df.empty:
            call_df['FH_OPEN_INT'] = call_df['FH_OPEN_INT'].astype(int)
            call_df['FH_TOT_TRADED_QTY'] = call_df['FH_TOT_TRADED_QTY'].astype(int)
            call_df = call_df.sort_values(by=['FH_OPEN_INT'], ascending=False)
            call_total_open_int = call_df['FH_OPEN_INT'].sum()
            call_total_volume = call_df['FH_TOT_TRADED_QTY'].sum()

            call_df['FH_OPEN_INT_PERCENTAGE'] = (call_df['FH_OPEN_INT'] / call_total_open_int) * 100
            call_df['FH_STRIKE_PRICE'] = call_df['FH_STRIKE_PRICE'].astype(float)
            call_df['ABS_DIFF'] = abs(call_df['FH_STRIKE_PRICE'] - float(underlying_value))

            sorted_call_df = call_df.sort_values(by='ABS_DIFF')
            closest_rows = sorted_call_df[['FH_STRIKE_PRICE', 'FH_OPEN_INT','ABS_DIFF','FH_OPEN_INT_PERCENTAGE','FH_TOT_TRADED_QTY']]

            call_df['FH_OPEN_INT'].idxmax()
            row_with_max_open_int = call_df.loc[call_df['FH_OPEN_INT'].idxmax()]
            row_with_max_open_int['FH_OPEN_INT']
            near_strike_call_open_int = closest_rows['FH_OPEN_INT'].head(3).sum()
            near_strike_call_vol = closest_rows['FH_TOT_TRADED_QTY'].head(3).sum()

          if not put_df.empty and not call_df.empty:
            pcr = float(put_total_open_int/call_total_open_int)
            pcr_vol = float(put_total_volume/call_total_volume)
            near_pcr = float(near_strike_put_open_int/near_strike_call_open_int)
            near_pcr_vol = float(near_strike_put_vol/near_strike_call_vol)
            new_data = {'Year': year,'Week_Number': week_num,'pcr': pcr,'pcr_vol': pcr_vol,'near_pcr': near_pcr,'near_pcr_vol': near_pcr_vol}
            print(new_data)
            pcr_data_df = pd.concat([pcr_data_df, pd.DataFrame([new_data])], ignore_index=True)
          # else:
          #   new_data = {'Year': year,'Week_Number': week_num,'pcr': None,'pcr_vol': None,'near_pcr': None,'near_pcr_vol': None}
          #   pcr_data_df = pd.concat([pcr_data_df, pd.DataFrame([new_data])], ignore_index=True)
      except:
          print("error occured")



29-03-2019 02-04-2019 04-Apr-2019 2019 14
11713.2
{'Year': 2019, 'Week_Number': 14, 'pcr': 1.1708342534866734, 'pcr_vol': 0.8627925922693085, 'near_pcr': 0.7943473486101075, 'near_pcr_vol': 1.2879005814291904}
05-04-2019 10-04-2019 11-Apr-2019 2019 15
11584.3
{'Year': 2019, 'Week_Number': 15, 'pcr': 0.7011844245950104, 'pcr_vol': 0.8253452969518855, 'near_pcr': 1.6614396920633079, 'near_pcr_vol': 1.2879005814291904}
12-04-2019 16-04-2019 18-Apr-2019 2019 16
11787.15
{'Year': 2019, 'Week_Number': 16, 'pcr': 1.632566080228602, 'pcr_vol': 1.0549640370580073, 'near_pcr': 0.6882046659853052, 'near_pcr_vol': 1.2879005814291904}
19-04-2019 24-04-2019 25-Apr-2019 2019 17
11726.15
{'Year': 2019, 'Week_Number': 17, 'pcr': 1.3235243961784657, 'pcr_vol': 0.8804065532325827, 'near_pcr': 0.27564291546166536, 'near_pcr_vol': 1.2879005814291904}
26-04-2019 30-04-2019 02-May-2019 2019 18

error occured
03-05-2019 08-05-2019 09-May-2019 2019 19
11359.45
{'Year': 2019, 'Week_Number': 19, 'pcr': 0.3183686

KeyboardInterrupt: 

In [ ]:
pcr_data_df

In [ ]:
pivoted_data_1.columns.name = None
pivoted_data_1.reset_index(inplace=True)


pivoted_data_2.columns.name = None
pivoted_data_2.reset_index(inplace=True)

pivoted_data_1['Friday'] = pivoted_data_1['Friday'].shift(1)

pivoted_data_1 = pivoted_data_1[['Year','Week_Number', 'Friday', 'Monday', 'Tuesday',  'Wednesday','Thursday']]
pivoted_data_1 = pivoted_data_1.dropna(thresh=pivoted_data_1.shape[1] - 5 + 1)
pivoted_data_1['Thursday'].fillna(pivoted_data_1['Wednesday'], inplace=True)

pivoted_data_2['Friday'] = pivoted_data_2['Friday'].shift(1)

pivoted_data_2 = pivoted_data_2[['Year','Week_Number', 'Friday', 'Monday', 'Tuesday',  'Wednesday','Thursday']]
pivoted_data_2 = pivoted_data_2.dropna(thresh=pivoted_data_2.shape[1] - 5 + 1)
pivoted_data_2['Thursday'].fillna(pivoted_data_2['Wednesday'], inplace=True)

In [ ]:
pivoted_data_2

In [ ]:
pivoted_data_2['Year_Week'] = pivoted_data_2['Year'].astype(str) + '_' + pivoted_data_2['Week_Number'].astype(str)
#pivoted_data_3 = pivoted_data_2[pivoted_data_2['Year'] == 2024]
pivoted_data_3 = pivoted_data_2
print(len(pivoted_data_3))
len(pivoted_data_3[(pivoted_data_3['Thursday'] < 0.5) & (pivoted_data_3['Thursday'] > -0.5)])

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(10, 5))
plt.plot(pivoted_data_3['Thursday'], marker='o', linestyle='-', color='b')
plt.xlabel('Year_Week')
plt.ylabel('Thursday')
plt.title('Thursday Values by Year_Week')
plt.xticks(rotation=45)
plt.grid(True)
plt.axhline(y=0.8, color='red', linestyle='--')
plt.axhline(y=-0.8, color='red', linestyle='--')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
historic_vix_df = pd.read_csv('/content/IndiaVIX_5Ydata.csv')
print(historic_vix_df.head(5))
historic_vix_df['Change %'] = historic_vix_df['Change %'].str.replace("%", "").astype(float)
historic_vix_df = historic_vix_df.rename(columns={'Change %': 'vix_change', 'Price': 'vix_close','Date ': 'Date'})
historic_vix_df = historic_vix_df[['Date', 'vix_change', 'vix_close']]
historic_vix_df.describe()